In [1]:
import ctypes
import numpy as np
from numba import cuda

In [2]:
# a=np.arange(6).reshape(2,3).astype(np.float32)
a = np.random.randn(100,100).astype(np.float32)
# ad=a.ravel('F')
a=np.asfortranarray(a)
a.nbytes/1024/1024

0.03814697265625

In [3]:
# b=np.arange(12).reshape(3,4).astype(np.float32)
b = np.random.randn(100,100).astype(np.float32)
# bd=b.ravel('F')
b=np.asfortranarray(b)
b.nbytes/1024/1024

0.03814697265625

In [4]:
c=np.empty((a.shape[0],b.shape[1]),dtype=np.float32,order='F')
# cd=c.ravel('F')
c=np.asfortranarray(c)
c.nbytes/1024/1024

0.03814697265625

In [5]:
a.shape,b.shape,c.shape

((100, 100), (100, 100), (100, 100))

In [10]:
sgemm=ctypes.CDLL('nnet/libsgemm.so')
al,bet,b0=1,0,np.array([0],dtype=np.float32)

In [11]:
b0

array([0.], dtype=float32)

In [12]:
%%time
da=cuda.to_device(a)
db=cuda.to_device(b)
# dc=cuda.to_device(cf)
dc=cuda.device_array_like(c)

CPU times: user 2.43 ms, sys: 507 µs, total: 2.93 ms
Wall time: 2.36 ms


In [13]:
m=a.shape[0]
n=b.shape[1]
k=b.shape[0]
lda=a.shape[0]
ldb=b.shape[0]
ldc=c.shape[0]
print(m,n,k)
print(lda,ldb,ldc)

100 100 100
100 100 100


In [14]:
da.shape,db.shape,dc.shape

((100, 100), (100, 100), (100, 100))

In [15]:
%%time
sgemm.createHandle()

CPU times: user 198 ms, sys: 62.4 ms, total: 261 ms
Wall time: 259 ms


0

In [16]:
%%time
sgemm.gemm(ord('N'),ord('N'),
           m,n,k,
           ctypes.c_float(al),
           da.device_ctypes_pointer, lda,
           db.device_ctypes_pointer, ldb,
           ctypes.c_float(bet),
           dc.device_ctypes_pointer, ldc,
           ctypes.c_void_p(b0.ctypes.data))

CPU times: user 323 µs, sys: 78 µs, total: 401 µs
Wall time: 333 µs


0

In [17]:
%%time
sgemm.destroyHandle()

CPU times: user 279 µs, sys: 66 µs, total: 345 µs
Wall time: 245 µs


0

In [18]:
%%time
d=a.dot(b)

CPU times: user 34 ms, sys: 1.14 ms, total: 35.2 ms
Wall time: 15.4 ms


In [19]:
cd=dc.copy_to_host().reshape(d.shape,order='A')

In [20]:
mcd=d-cd
abs(mcd).sum(),abs(mcd).mean(),abs(mcd).max()

(0.01379291, 1.379291e-06, 1.5258789e-05)

In [35]:
mcd

array([[-9.5367432e-07, -1.9073486e-06,  3.8146973e-06, ...,
         4.7683716e-07,  7.1525574e-07,  0.0000000e+00],
       [-2.6226044e-06, -1.9073486e-06,  1.9073486e-06, ...,
         3.8146973e-06,  2.3841858e-07,  1.9073486e-06],
       [-2.3841858e-06,  4.7683716e-07,  2.3841858e-06, ...,
         7.1525574e-07, -9.5367432e-07,  0.0000000e+00],
       ...,
       [ 2.8610229e-06,  1.9073486e-06,  3.2782555e-07, ...,
        -9.5367432e-07, -1.9073486e-06,  1.9073486e-06],
       [-1.9073486e-06, -4.7683716e-07, -4.7683716e-07, ...,
         9.5367432e-07, -2.3841858e-06, -1.4305115e-06],
       [-2.8610229e-06,  0.0000000e+00, -1.9073486e-06, ...,
        -1.1920929e-07, -2.3841858e-07,  3.8146973e-06]], dtype=float32)

In [31]:
(cd.ravel('K')-d.ravel('K'))[:100]

array([ 9.53674316e-07, -1.48842688e+01, -2.64646473e+01, -1.00968142e+01,
       -5.83914280e+00,  1.72445908e+01, -3.18626356e+00,  1.07400637e+01,
        3.34959412e+00, -6.38924742e+00, -4.84065151e+00,  1.43555031e+01,
       -2.08206444e+01, -2.00208569e+01, -1.34401178e+01,  2.43318100e+01,
        1.77149487e+00,  1.10745869e+01, -3.41328049e+01, -9.46953964e+00,
       -1.13147974e+01,  8.39394379e+00,  3.38804893e+01, -9.31696177e-01,
        1.56494408e+01,  3.70879478e+01,  1.57008915e+01, -2.19927883e+01,
       -2.88996792e+00, -1.14193726e+01, -3.75335956e+00, -1.73858719e+01,
       -2.08504105e+01, -2.00567722e-01, -1.28826904e+01, -4.89148521e+00,
       -8.56719780e+00, -4.71271181e+00,  1.20205288e+01,  3.79198790e+00,
       -1.20845747e+00, -2.79968357e+00,  5.16406155e+00, -7.06129789e+00,
        1.02989769e+01,  5.64295387e+00, -3.52496386e+00, -1.70666993e+00,
       -1.17164147e+00, -1.23304367e+01, -1.46896877e+01,  2.35910301e+01,
        2.70744538e+00,  

In [42]:
np.allclose(d,cd)

True

In [19]:
gg=np.empty((11900,20000),dtype=np.float32)
gg[:]=np.random.rand(*gg.shape)
gg.nbytes/1024/1024

KeyboardInterrupt: 

In [ ]:
%%time
dgg=cuda.to_device(gg)#.T.reshape(gg.shape)

In [ ]:
%%time
dgt=cuda.device_array_like(gg)

In [ ]:
sgemm=ctypes.CDLL('nnet/libsgemm.so')
al,bet=1,0
m,n=gg.shape

In [ ]:
%%timeit
sgemm.createHandle()
sgemm.transpose(dgg.device_ctypes_pointer,
                dgt.device_ctypes_pointer,
                ctypes.c_int(n),ctypes.c_int(m),
                ctypes.c_float(al),ctypes.c_float(bet))
sgemm.destroyHandle()

In [ ]:
%%time
(dgt.copy_to_host().ravel('K')==gg.ravel('F')).all()

In [ ]:
%%time
gg.ravel('F')

In [ ]:
del dgg
del dgt